In [28]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv("../dataset/AND_dataset/AND_dataset/AND_features.csv")
data = data.iloc[:,1:]
data

,ImageId,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,0359a,2,1,1,0,2,2,0,2,2
1,0577a,2,1,1,0,2,2,0,1,2
2,1120a,2,1,1,3,2,2,0,2,2
3,1120b,1,1,1,0,2,2,0,2,2
4,1120c,2,1,1,0,2,2,0,0,2
5,1121a,2,1,1,3,2,2,0,1,2
6,1121b,2,1,1,0,2,2,0,3,2
7,1121c,1,1,1,0,2,2,0,1,2
8,1229b,1,1,1,3,2,2,0,2,2
9,1302a,2,1,1,3,2,2,0,0,2


In [25]:
data["ImageId"] = data.ImageId.str.split("[a-z]",expand=True)
data

,ImageId,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,0359,2,1,1,0,2,2,0,2,2
1,0577,2,1,1,0,2,2,0,1,2
2,1120,2,1,1,3,2,2,0,2,2
3,1120,1,1,1,0,2,2,0,2,2
4,1120,2,1,1,0,2,2,0,0,2
5,1121,2,1,1,3,2,2,0,1,2
6,1121,2,1,1,0,2,2,0,3,2
7,1121,1,1,1,0,2,2,0,1,2
8,1229,1,1,1,3,2,2,0,2,2
9,1302,2,1,1,3,2,2,0,0,2


462